In [1]:
import re
import string
str_punc = string.punctuation.replace(',', '').replace("'",'')

def clean(text):
    global str_punc
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = text.lower()
    return text   

In [2]:
import pandas as pd
import sklearn.pipeline

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Read in training data set
train_df = pd.read_csv('./data/train.txt', names=['Text', 'Emotion'], sep=';')
train_df['Text'] = train_df['Text'].apply(clean) 
# Categorical data type for truth labels
train_df['Emotion'] = le.fit_transform(train_df['Emotion'])

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
pipeline = Pipeline(
[
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(
        loss='modified_huber',
        penalty='l2',
        alpha=1e-3,
        random_state=42,
        max_iter=100,
        tol=None,
    )),
]
)
# Train model
classifier = pipeline.fit(train_df['Text'], train_df['Emotion'])

In [4]:
le = LabelEncoder()
# Read in training data set
test_df = pd.read_csv('./data/test.txt', names=['Text', 'Emotion'], sep=';')
test_df['Text'] = test_df['Text'].apply(clean) 
# Categorical data type for truth labels
test_df['Emotion'] = le.fit_transform(test_df['Emotion'])

In [5]:
acc=classifier.predict(test_df['Text'])

In [6]:
from sklearn.metrics import f1_score, accuracy_score
print(accuracy_score(test_df['Emotion'],acc))

0.8505


In [7]:
import pickle
pickle.dump(classifier, open('supportVectorMachine.pkl','wb'))